# Оптимизация гиперпараметров моделей в scikit-learn

Оптимизация гиперпараметров — это важный шаг в машинном обучении, который помогает улучшить производительность модели на тестовых данных. 
В этом ноутбуке мы:
1. Изучим использование **GridSearchCV**.
2. Реализуем перебор параметров вручную.
3. Рассмотрим альтернативные подходы, такие как **RandomizedSearchCV**.


## Генерация данных

In [6]:
from sklearn.datasets import make_regression
from sklearn.model_selection import train_test_split

# Генерация синтетического набора данных
X, y = make_regression(n_samples=200, n_features=3, noise=10, random_state=42)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print('Размер обучающей выборки:', X_train.shape)
print('Размер тестовой выборки:', X_test.shape)

Размер обучающей выборки: (160, 3)
Размер тестовой выборки: (40, 3)


## GridSearchCV

In [7]:
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error

# Сетка гиперпараметров
param_grid = {
    'alpha': [0.1, 1.0, 10.0],
    'fit_intercept': [True, False]
}

# GridSearchCV
grid_search = GridSearchCV(Ridge(), param_grid, cv=5, scoring='neg_mean_squared_error')
grid_search.fit(X_train, y_train)

print('Лучшие параметры:', grid_search.best_params_)
print('Лучший результат (neg_mean_squared_error):', grid_search.best_score_)

# Оценка на тестовых данных
best_model = grid_search.best_estimator_
y_pred = best_model.predict(X_test)
mse_test = mean_squared_error(y_test, y_pred)
print('MSE на тестовых данных:', mse_test)

Лучшие параметры: {'alpha': 0.1, 'fit_intercept': False}
Лучший результат (neg_mean_squared_error): -107.92350419524944
MSE на тестовых данных: 131.2267578837927


## Реализация GridSearchCV вручную

In [8]:
from sklearn.model_selection import KFold

# Реализация руками
kf = KFold(n_splits=5, shuffle=True, random_state=42)
best_params = None
best_mse = float('inf')

for alpha in [0.1, 1.0, 10.0]:
    for fit_intercept in [True, False]:
        mse_scores = []
        for train_idx, val_idx in kf.split(X_train):
            X_tr, X_val = X_train[train_idx], X_train[val_idx]
            y_tr, y_val = y_train[train_idx], y_train[val_idx]
            model = Ridge(alpha=alpha, fit_intercept=fit_intercept)
            model.fit(X_tr, y_tr)
            y_pred = model.predict(X_val)
            mse_scores.append(mean_squared_error(y_val, y_pred))
        avg_mse = sum(mse_scores) / len(mse_scores)
        print(f'alpha={alpha}, fit_intercept={fit_intercept}, MSE={avg_mse}')
        if avg_mse < best_mse:
            best_mse = avg_mse
            best_params = {'alpha': alpha, 'fit_intercept': fit_intercept}

print('Лучшие параметры (ручной перебор):', best_params)

alpha=0.1, fit_intercept=True, MSE=108.48284280092784
alpha=0.1, fit_intercept=False, MSE=107.94946185904796
alpha=1.0, fit_intercept=True, MSE=109.18217563269107
alpha=1.0, fit_intercept=False, MSE=108.68813800377514
alpha=10.0, fit_intercept=True, MSE=172.54703849711206
alpha=10.0, fit_intercept=False, MSE=170.768775289724
Лучшие параметры (ручной перебор): {'alpha': 0.1, 'fit_intercept': False}


## RandomizedSearchCV

In [9]:
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import uniform

# RandomizedSearchCV
param_distributions = {
    'alpha': uniform(0.1, 10.0),
    'fit_intercept': [True, False]
}

random_search = RandomizedSearchCV(
    Ridge(), param_distributions, n_iter=20, cv=5, scoring='neg_mean_squared_error', random_state=42
)
random_search.fit(X_train, y_train)

print('Лучшие параметры (RandomizedSearchCV):', random_search.best_params_)

Лучшие параметры (RandomizedSearchCV): {'alpha': np.float64(0.10778765841014329), 'fit_intercept': False}


## Заключение

В этом ноутбуке мы:
- Изучили использование **GridSearchCV** и реализовали его вручную.
- Рассмотрели альтернативный метод **RandomizedSearchCV**.

Каждый метод имеет свои плюсы и минусы:
- GridSearchCV подходит для небольших сеток гиперпараметров.
- RandomizedSearchCV быстрее для больших пространств гиперпараметров.

Выбор метода зависит от задачи, доступных ресурсов и размера пространства гиперпараметров.